# Homework 2-Question-2 : Construction with constraints

 During the next 4 months, a construction firm must
complete three projects. Each project has a deadline as well as labor requirements. Project 1 must be
completed no later than 3 months from now and requires 8 worker-months of labor. Project 2 must
be completed no later than 4 months from now and requires 10 worker-months of labor. Project 3
must be completed no later than 2 months from now and requires 12 worker-months of labor. Each
month, 8 workers are available. During a given month, no more than 6 workers can work on a single
job. Determine whether all three projects can be completed on time.

## Problem Data 

In [5]:
n_months = 4
n_projects = 3
tasks = range(1,n_projects)

deadlines = Dict(zip(tasks,[3, 4, 2]))   # deadlines of each project
work_required = Dict(zip(tasks,[8, 10, 12]))   # Total worker-months required for each project.
;

## Problem Model 

In [6]:
using JuMP

m = Model()
# variables for work required for a project to complete at the start of each month
# (Project, month)
@variable(m, workRequired[1:n_projects,1:n_months+1] >= 0)
#Initial condition to put total worker months required to be equal to work required for a project to
#complete at the start of 1st month.
for p in 1:n_projects 
    @constraint(m, workRequired[p,1] == work_required[p])
end
#variable to provide labor to each project each month
@variable(m, 0 <= laborProvider[1:n_projects,1:n_months] <= 6)
#constraint to limit the total availability of workers to 8 per month.
@constraint(m, worker_constr[i in 1:n_months], sum(laborProvider[:,i]) <= 8)
#constraints for flow of work left from month to next month
for p in 1:n_projects
    for mo in 2:n_months+1
        @constraint(m, workRequired[p,mo] + laborProvider[p,mo-1] == workRequired[p,mo-1])
    end
end
# objective to minimize the work required for completion of any project after its deadline.
# A min value of Zero would tell us that its possible to deliver on all of these projects.
@objective(m, Min, (sum(sum(workRequired[p, deadlines[p]+1:n_months+1]) for p in 1:n_projects)))

workRequired[1,4] + workRequired[1,5] + workRequired[2,5] + workRequired[3,3] + workRequired[3,4] + workRequired[3,5]

In [7]:
solve(m)
delayInCompletion = getobjectivevalue(m)
println("Delay in completion of projects: ",delayInCompletion)
laborProvided = getvalue(laborProvider)
println("Work done monthly for Projects")
for p in 1:n_projects
    println("Project ",p," (",sum(laborProvided[p,:]),"): ", laborProvided[p,:])
end

Delay in completion of projects: 0.0
Work done monthly for Projects
Project 1 (8.0): [2.0,0.0,6.0,0.0]
Project 2 (10.0): [0.0,2.0,2.0,6.0]
Project 3 (12.0): [6.0,6.0,0.0,0.0]


The delay in completion of projects is equal to Zero, which means all the projects met their individual deadlines and were completed in the required 4 months.